## softmax_cross_entropy_with_logits

#### keywords : cross_entropy, one_hot, reshape

### 예제 1. Animal Classification

In [2]:
import tensorflow as tf
import numpy as np

In [8]:
xy = np.loadtxt('data-04-zoo.csv', delimiter=',', dtype=np.float64)
x_data = xy[:, :-1]
y_data = xy[:, [-1]]

In [9]:
x_data.shape, y_data.shape

((101, 16), (101, 1))

In [13]:
# y_data => Y_one_hot  (이 방법도 있으나 tensorflow에는 tf.one_hot 이 있음)
from sklearn.preprocessing import OneHotEncoder
Y_one_hot = OneHotEncoder().fit_transform(y_data)
Y_one_hot = Y_one_hot.toarray()
Y_one_hot[:5]

/Users/imjunghee/anaconda3/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:415: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


array([[1., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0.]])

In [24]:
tf.reset_default_graph()
nb_classes = 7
# Placeholders : tf.one_hot and tf.reshape
X = tf.placeholder(tf.float32, shape=[None, 16])
Y = tf.placeholder(tf.int32, shape=[None, 1])
Y_one_hot = tf.one_hot(Y, nb_classes)  # one hot shape = (?, 1, 7) 3차원
Y_one_hot = tf.reshape(Y_one_hot, [-1, nb_classes])  #2차원으로 변환


# Variables
W = tf.Variable(tf.random_normal([16, nb_classes]), name='weight')
b = tf.Variable(tf.random_normal([nb_classes]), name='bias')

# logits and hypothesis
logits = tf.matmul(X, W) + b
hypothesis = tf.nn.softmax(logits)

# cross entropy cost/loss
cost_i = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y_one_hot)
cost = tf.reduce_mean(cost_i)

# train / Minimize
train = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

# accuracy computation
prediction = tf.argmax(hypothesis, 1)
correct_prediction = tf.equal(prediction, tf.argmax(Y_one_hot, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# Launch the graph
sess = tf.Session()
sess.run(tf.global_variables_initializer())

for step in range(2000):
    cost_val, _ = sess.run([cost, train], feed_dict={X: x_data, Y: y_data})
    if step % 200 == 0:
        loss, acc = sess.run([cost, accuracy], feed_dict={X: x_data, Y: y_data})
        print("Step: {:5d}\t Loss: {:3f}\t Acc: {:.2%}".format(step, loss, acc))

Step:     0	 Loss: 5.375578	 Acc: 9.90%
Step:   200	 Loss: 0.422413	 Acc: 87.13%
Step:   400	 Loss: 0.244148	 Acc: 94.06%
Step:   600	 Loss: 0.171678	 Acc: 96.04%
Step:   800	 Loss: 0.131683	 Acc: 99.01%
Step:  1000	 Loss: 0.106539	 Acc: 99.01%
Step:  1200	 Loss: 0.089446	 Acc: 100.00%
Step:  1400	 Loss: 0.077136	 Acc: 100.00%
Step:  1600	 Loss: 0.067860	 Acc: 100.00%
Step:  1800	 Loss: 0.060620	 Acc: 100.00%


In [25]:
# predict
pred = sess.run(prediction, feed_dict={X: x_data})  # pred.shape = (101,)
# y_data: (N, 1) = flatten() => (N, ) matches pred.shape, y_data.shape = (101, 1)
for p, y in zip(pred, y_data.flatten()):
    print("[{}] Prediction: {} True Y: {}".format(p == int(y), p, int(y)))  # if equal True if not False, pred 라벨 값, 실제 라벨 값

[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 3 True Y: 3
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 3 True Y: 3
[True] Prediction: 3 True Y: 3
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 1 True Y: 1
[True] Prediction: 3 True Y: 3
[True] Prediction: 6 True Y: 6
[True] Prediction: 6 True Y: 6
[True] Prediction: 6 True Y: 6
[True] Prediction: 1 True Y: 1
[True] Prediction: 0 True Y: 0
[True] Prediction: 3 True Y: 3
[True] Prediction: 0 True Y: 0
[True] Prediction: 1 True Y: 1
[True] Prediction: 1 True Y: 1
[True] Prediction: 0 True Y: 0
[True] Prediction: 1 True Y: 1
[True] Prediction: 5 True Y: 5
[True] Prediction: 4 True Y: 4
[True] Prediction: 4 True Y: 4
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 0 True Y: 0
[True] Prediction: 5 True Y: 5
[True] Prediction: 0 True Y: 0
[True] P

In [28]:
y_data[:5]

array([[0.],
       [0.],
       [3.],
       [0.],
       [0.]])

In [29]:
y_data[:5].flatten()

array([0., 0., 3., 0., 0.])

In [30]:
pred.shape

(101,)